## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split

## Data Preparation

In [13]:
df=pd.read_csv("housing.csv")
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [14]:
df.replace([np.inf, -np.inf], np.nan)
df.dropna(inplace=True) 

In [15]:
targets = df[["median_house_value"]].to_numpy()
inputs = df.drop(["median_house_value", 'ocean_proximity'], axis=1).to_numpy()  

In [16]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

In [17]:
inputs_mean = torch.mean(inputs, axis=0)
targets_mean = torch.mean(targets, axis=0)
inputs = inputs / inputs_mean  
targets = targets / targets_mean 

In [18]:
dataset = TensorDataset(inputs, targets)

In [19]:
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

## Define Two Models

In [20]:
model1 = nn.Sequential(nn.Linear(8, 4),
                      nn.ReLU(),

                      nn.Linear(4, 2),
                      nn.Softmax(),

                      
                      nn.Linear(2, 1),
                      nn.Sigmoid())

model2 = nn.Sequential(nn.Linear(8, 4),
                      nn.Softmax(),

                      nn.Linear(4, 1),
                      nn.ReLU())

In [25]:
# Utility function to train the model
def fit(num_epochs, model, criterion, optimizer, train_loader):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_loader:
            # 1. Generate predictions
            pred = model(xb)
            # 2. Calculate loss
            loss = criterion(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            optimizer.step()
            # 5. Reset the gradients to zero
            optimizer.zero_grad()
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

## Fit Model1

In [26]:
# Define loss function
criterion = F.mse_loss
# Define optimizer
optimizer = torch.optim.SGD(model1.parameters(), lr=1e-4)
fit(100, model1, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 0.2315
Epoch [20/100], Loss: 0.4351
Epoch [30/100], Loss: 0.2716
Epoch [40/100], Loss: 0.4443
Epoch [50/100], Loss: 0.8036
Epoch [60/100], Loss: 0.4235
Epoch [70/100], Loss: 0.2056
Epoch [80/100], Loss: 0.4862
Epoch [90/100], Loss: 0.1465
Epoch [100/100], Loss: 0.2487


## Fit Model1 with Regularization

In [27]:
# Define loss function
criterion = F.mse_loss
# Define optimizer
optimizer = torch.optim.SGD(model1.parameters(),weight_decay=2, lr=1e-4)
fit(100, model1, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 0.5234
Epoch [20/100], Loss: 0.2387
Epoch [30/100], Loss: 0.4271
Epoch [40/100], Loss: 0.3930
Epoch [50/100], Loss: 0.5617
Epoch [60/100], Loss: 0.3767
Epoch [70/100], Loss: 0.3616
Epoch [80/100], Loss: 0.4181
Epoch [90/100], Loss: 0.5975
Epoch [100/100], Loss: 0.1678


## Fit Model2

In [28]:
# Define loss function
criterion = F.mse_loss
# Define optimizer
optimizer = torch.optim.SGD(model2.parameters(), lr=1e-4)
fit(100, model2, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 1.7673
Epoch [20/100], Loss: 1.8055
Epoch [30/100], Loss: 1.2797
Epoch [40/100], Loss: 1.8131
Epoch [50/100], Loss: 1.3826
Epoch [60/100], Loss: 1.1318
Epoch [70/100], Loss: 0.8978
Epoch [80/100], Loss: 1.0484
Epoch [90/100], Loss: 1.1879
Epoch [100/100], Loss: 1.7616


## Fit Model2 with Regularization

In [29]:
# Define loss function
criterion = F.mse_loss
# Define optimizer
optimizer = torch.optim.SGD(model2.parameters(),weight_decay=2, lr=1e-4)
fit(100, model2, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 1.0924
Epoch [20/100], Loss: 1.1683
Epoch [30/100], Loss: 1.4764
Epoch [40/100], Loss: 1.6601
Epoch [50/100], Loss: 1.2408
Epoch [60/100], Loss: 1.6125
Epoch [70/100], Loss: 0.8963
Epoch [80/100], Loss: 1.6152
Epoch [90/100], Loss: 0.8306
Epoch [100/100], Loss: 1.8554
